In [ ]:
import numpy as np
import pandas as pd
import itertools
import h5py
import matplotlib.pyplot as plt
from scipy import stats
from scipy import special
from scipy import integrate
from scipy import interpolate
from scipy import linalg
from scipy import signal
from scipy.optimize import curve_fit
import time
from pathlib import Path
import os
import random
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel

plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['figure.dpi'] = 120
#plt.rcParams['text.usetex'] = True

import sys
import mpld3

#mpld3.enable_notebook()

sys.path.insert(0, '..')
from modules import prh
from modules import regression as rg
from modules import utils

## Load data

In [ ]:
he_qso = pd.read_csv('../data/mindstep/HE_0047-1756.csv', delim_whitespace=True)
he_qso = he_qso.dropna()

t = he_qso['MJD'].values
magAV = he_qso['magAV'].values
magBV = he_qso['magBV'].values
sigmaAV = he_qso['sigmaAV'].values
sigmaBV = he_qso['sigmaBV'].values
magAR = he_qso['magAR'].values
magBR = he_qso['magBR'].values
sigmaAR = he_qso['sigmaAR'].values
sigmaBR = he_qso['sigmaBR'].values

## Select curves to analyze

In [ ]:
which = 'magV'

# V-band
if which == 'magV':
    y1 = magAV
    y2 = magBV
    err1 = sigmaAV
    err2 = sigmaBV
    label1 = 'magAV'
    label2 = 'magBV'

# R-band
elif which == 'magR':
    y1 = magAR
    y2 = magBR
    err1 = sigmaAR
    err2 = sigmaBR
    label1 = 'magAR'
    label2 = 'magBR'


# ratio V/R
elif which == 'magV_su_magR':
    y1 = magAV / magAR
    y2 = magBV / magBR
    err1 = (sigmaAV/magAV + sigmaAR/magAR) * y1
    err2 = (sigmaBV/magBV + sigmaBR/magBR) * y2
    label1 = 'magAV/magAR'
    label2 = 'magBV/magBR'
    
elif which == 'magR_su_magV':
    y1 = magAR / magAV
    y2 = magBR / magBV
    err1 = (sigmaAV/magAV + sigmaAR/magAR) * y1
    err2 = (sigmaBV/magBV + sigmaBR/magBR) * y2
    label1 = 'magAR/magAV'
    label2 = 'magBR/magBV'
    
# diff V - R
elif which == 'magV_meno_magR':
    y1 = magAV - magAR
    y2 = magBV - magBR
    err1 = (sigmaAV + sigmaAR)
    err2 = (sigmaBV + sigmaBR)
    label1 = 'magAV - magAR'
    label2 = 'magBV - magBR'
     


In [ ]:
season_masks = utils.get_season_masks(t)

#plt.scatter(t, y1, label='all')
plt.title(label1)
for i, mask in enumerate(season_masks):
    plt.scatter(t[mask], y1[mask], label='season %s' % (i+1))
plt.legend()

In [ ]:
kernel = ConstantKernel(2, (1e-3, 1e2)) * Matern(length_scale=200.0, length_scale_bounds=(1, 300), nu=1.5)

gp1 = rg.fit_GP_to_lightcurve(t, y1, err1, kernel)
gp2 = rg.fit_GP_to_lightcurve(t, y2, err2, kernel)

gp_step = 0.2
# number of points to add on the left and right ends of the domain in order to make possible to shift the signals
dt_ext = 100
support = np.arange(t[0] - dt_ext, t[-1] + dt_ext, gp_step)

ypred1, sigma1 = gp1.predict(np.expand_dims(support, 1), return_std=True)
ypred2, sigma2 = gp2.predict(np.expand_dims(support, 1), return_std=True)

CL = 0.95
fig = plt.figure()
plt.scatter(t, y1, label=label1, color='b')
plt.scatter(t, y2, label=label2, color='g')

plt.plot(support, ypred1, label='%s GP prediction' % label1, color = 'b')
plt.plot(support, ypred2, label='%s GP prediction' % label2, color = 'g')

plt.fill_between(support, ypred1 - special.erfinv(CL)*sigma1, ypred1 + special.erfinv(CL)*sigma1,
         alpha=.5, fc='b', ec='None', label=r'95\% confidence interval A')
plt.fill_between(support, ypred2 - special.erfinv(CL)*sigma2, ypred2 + special.erfinv(CL)*sigma2,
         alpha=.5, fc='g', ec='None', label=r'95\% confidence interval B')
plt.legend(fontsize=10)

plt.xlabel('t[MJD]')
plt.ylabel('mag')
plt.title('HE0047-1756 (MindStep data)')

fig.savefig('mindstep_gp_prediction.pdf')

In [ ]:
season_begs = utils.get_season_begs(t)
season_ends = utils.get_season_ends(t)

t_begs = t[season_begs]
t_ends = t[season_ends]

In [ ]:
n_p = 0
for beg, end in zip(t_begs, t_ends):
    n_p += len(support[(support >= beg) & (support <= end)])

missing_fraction = (len(support) - n_p) / (len(support))

print(f'Fraction of new points falling into season gaps: {missing_fraction:.3f}')

In [ ]:
def numeric_derivative(f: np.ndarray, step: float) -> np.ndarray:
    return (f[1:] - f[:-1]) / step


def WAV(f, sigma, step):
    f_prime = numeric_derivative(f, step)
    weights = 2 / (sigma[:-1] + sigma[1:])
    WAV = np.dot(np.abs(f_prime), weights) / weights.sum()
    return WAV

dt_min_days =  0
dt_max_days =  100

shift = np.arange(int(dt_min_days / gp_step),
                  int(dt_max_days / gp_step), 1)
win = int(dt_ext/gp_step)

WAV_values = []
for i in shift:
    diff = ypred1[win:-win] - ypred2[win + i:-win + i]
    sigma_diff = sigma1[win:-win] + sigma2[win + i:-win + i]
    WAV_values.append(WAV(diff, sigma_diff, gp_step))
    

delay_idx = np.argmin(WAV_values)
delay = shift[delay_idx] * gp_step
print(delay)

shift_dom = gp_step*shift
fig = plt.figure()
plt.plot(shift_dom, WAV_values, label='WAV loss function')
plt.xlabel('time lag[days]')
plt.axvline(x=7.6, color='red', label='Mindstep result (7.6 days)')
plt.legend()
plt.title('WAV %s' % which)
fig.savefig('wav_loss_%s_maxdelay_%s.pdf' % (which, dt_max_days))

In [ ]:
fig, axs = plt.subplots(figsize=(15, 12), nrows=2, ncols=1)

best_shift = shift[delay_idx]
axs[0].plot(ypred1[win:-win], label=label1)


plot_shifts = [shift[0],  
               shift[delay_idx]]

for dt_idx in plot_shifts:
    axs[0].plot(ypred2[win + dt_idx :-win + dt_idx], 
                label='%s shifted by %.4f days' % (label2, shift[dt_idx]*gp_step))
    axs[1].plot(ypred1[win:-win] - ypred2[win + dt_idx :-win + dt_idx], 
                label='dt = %.4f, WAV = %.4f' % (shift[dt_idx]*gp_step, WAV_values[dt_idx]))

axs[0].legend()
axs[1].legend()

axs[0].set_title('GP predicted light curves')
axs[1].set_title('Difference between the shifted curves')

fig.savefig('%s_various_shifts.pdf' % which)